# MOSFET

## Metal-Oxide-Semiconductor Field-Effect Transistor

The MOSFET is the fundamental building block of modern digital electronics. Understanding its operation is essential for circuit design and device engineering.

**Learning Objectives:**
- Understand MOSFET structure and operation
- Simulate transfer characteristics (Id-Vg) with PADRE
- Simulate output characteristics (Id-Vd) with PADRE
- Extract threshold voltage and transconductance from simulation data

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## 1. MOSFET Physics

### 1.1 Structure

```
     Gate (Metal/Poly)
     ================
     |   Gate Oxide  |
     ================
  Source    Channel    Drain
  (N+)    (Inversion)   (N+)
  ====     -------     ====
    |                    |
    |    P-substrate     |
    +--------------------+
           Body
```

### 1.2 Key Parameters

- **Threshold voltage** ($V_{th}$): Gate voltage to create inversion
  $$V_{th} = V_{FB} + 2\phi_F + \frac{\sqrt{2\epsilon_s q N_a (2\phi_F)}}{C_{ox}}$$

- **Drain current** (linear region, $V_{ds} < V_{gs} - V_{th}$):
  $$I_d = \mu_n C_{ox} \frac{W}{L} \left[(V_{gs} - V_{th})V_{ds} - \frac{V_{ds}^2}{2}\right]$$

- **Drain current** (saturation, $V_{ds} > V_{gs} - V_{th}$):
  $$I_d = \frac{1}{2} \mu_n C_{ox} \frac{W}{L} (V_{gs} - V_{th})^2$$

- **Transconductance**:
  $$g_m = \frac{\partial I_d}{\partial V_{gs}} = \mu_n C_{ox} \frac{W}{L} (V_{gs} - V_{th})$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_mosfet

# Physical constants
q = 1.6e-19
eps_0 = 8.85e-14  # F/cm
eps_si = 11.7 * eps_0
eps_ox = 3.9 * eps_0
kT = 0.0259  # eV at 300K
ni = 1.5e10  # intrinsic carrier concentration (cm^-3)

# MOSFET parameters
L = 0.1e-4    # Channel length (cm) = 100nm
W = 1e-4      # Channel width (cm) = 1um
tox = 5e-7    # Oxide thickness (cm) = 5nm
Na = 1e18     # Substrate doping (cm^-3)
mu_n = 400    # Electron mobility (cm^2/V-s)

# Calculated parameters
Cox = eps_ox / tox  # Oxide capacitance (F/cm^2)
phi_F = kT * np.log(Na / ni)  # Fermi potential

print("MOSFET Parameters:")
print("="*40)
print(f"Channel length: {L*1e4*1000:.0f} nm")
print(f"Oxide thickness: {tox*1e7:.0f} nm")
print(f"Cox = {Cox*1e6:.2f} μF/cm²")
print(f"phi_F = {phi_F:.3f} V")

---

## 2. Creating and Running a MOSFET Simulation

Let's create an NMOS transistor and run PADRE to examine its characteristics.

In [ ]:
# Create NMOS transistor
sim_nmos = create_mosfet(
    # Geometry
    channel_length=0.1,         # 100nm gate length
    gate_oxide_thickness=0.005, # 5nm oxide
    junction_depth=0.02,        # 20nm junction depth
    device_width=0.125,         # Device width
    device_depth=0.068,         # Substrate depth
    
    # Mesh
    nx=51,
    ny=51,
    
    # Doping
    channel_doping=1e18,        # Channel doping (p-type for NMOS)
    substrate_doping=5e16,      # Substrate doping
    source_drain_doping=1e20,   # N+ S/D doping
    device_type='nmos',
    
    # Models
    temperature=300,
    bgn=True,                   # Band-gap narrowing
    
    # Output - enable band diagrams
    log_bands_eq=True
)

print("NMOS Transistor Configuration:")
print("="*40)
print("Channel length: 100 nm")
print("Gate oxide: 5 nm")
print("Channel doping: 1e18 cm⁻³")
print("S/D doping: 1e20 cm⁻³")

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_nmos.generate_deck())

In [ ]:
# Run equilibrium simulation
print("Running MOSFET equilibrium simulation...")
result_eq = sim_nmos.run()

if result_eq.returncode == 0:
    print("Simulation completed successfully!")
    print(f"Output directory: {sim_nmos.working_dir}")
else:
    print("Simulation failed!")
    print(result_eq.stderr)

---

## 3. Transfer Characteristics (Id-Vg)

The transfer characteristic shows drain current vs gate voltage at fixed drain voltage. Let's simulate this with PADRE.

In [ ]:
# Create MOSFET with transfer characteristic sweep
sim_idvg = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    junction_depth=0.02,
    device_width=0.125,
    device_depth=0.068,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='nmos',
    
    # Models
    temperature=300,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="idvg",
    
    # Transfer characteristic: sweep Vgs at fixed Vds
    vgs_sweep=(0.0, 1.5, 0.05),  # Vgs: 0 to 1.5V
    vds=0.1                       # Vds = 0.1V (linear region)
)

print("Transfer Characteristic Simulation Configuration:")
print("="*40)
print("Gate voltage sweep: 0V to 1.5V")
print("Drain voltage: 0.1V (linear region)")
print(f"Number of bias points: {int(1.5/0.05) + 1}")

In [ ]:
# Run the transfer characteristic simulation
print("Running transfer characteristic simulation...")
print("(This may take a few minutes)")
result_idvg = sim_idvg.run()

if result_idvg.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")
    print(result_idvg.stderr)

In [ ]:
# Plot the simulated transfer characteristic
try:
    # Get I-V data from simulation
    iv_data = sim_idvg.get_iv_data()
    
    # Gate electrode is typically electrode 3, drain is electrode 2
    Vgs_sim, Id_sim = iv_data.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
    
    Vgs_sim = np.array(Vgs_sim)
    Id_sim = np.array(Id_sim)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Linear scale plot
    ax1.plot(Vgs_sim, Id_sim * 1e6, 'b-o', linewidth=2, markersize=4, label='PADRE Simulation')
    ax1.set_xlabel('Gate Voltage Vgs (V)', fontsize=12)
    ax1.set_ylabel('Drain Current Id (μA)', fontsize=12)
    ax1.set_title('Transfer Characteristic (Linear Scale)', fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.set_xlim(0, 1.5)
    
    # Log scale plot
    ax2.semilogy(Vgs_sim, Id_sim, 'b-o', linewidth=2, markersize=4, label='PADRE Simulation')
    ax2.set_xlabel('Gate Voltage Vgs (V)', fontsize=12)
    ax2.set_ylabel('Drain Current Id (A)', fontsize=12)
    ax2.set_title('Transfer Characteristic (Log Scale)', fontsize=14)
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_xlim(0, 1.5)
    
    # Add annotation for subthreshold slope
    ax2.annotate('Subthreshold\nregion', xy=(0.2, 1e-10), fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print("\nSimulated Transfer Characteristic:")
    print("="*40)
    print(f"Vgs range: {Vgs_sim[0]:.2f}V to {Vgs_sim[-1]:.2f}V")
    print(f"Id range: {Id_sim.min():.3e} A to {Id_sim.max():.3e} A")
    
except Exception as e:
    print(f"Could not plot transfer characteristic: {e}")
    print("Trying built-in plot method...")
    sim_idvg.plot_transfer(gate_electrode=3, drain_electrode=2)

In [ ]:
# Extract threshold voltage from simulation
try:
    # Method 1: Linear extrapolation
    # Find max gm point (maximum slope)
    gm = np.gradient(Id_sim, Vgs_sim)
    max_gm_idx = np.argmax(gm)
    
    if max_gm_idx > 0 and max_gm_idx < len(Id_sim) - 1:
        slope = gm[max_gm_idx]
        intercept = Id_sim[max_gm_idx] - slope * Vgs_sim[max_gm_idx]
        Vth_linear = -intercept / slope if slope > 0 else np.nan
    else:
        Vth_linear = np.nan
    
    # Method 2: Constant current method (Ith = 100nA)
    I_th = 1e-7  # 100 nA
    idx_above = np.where(Id_sim > I_th)[0]
    if len(idx_above) > 0:
        Vth_const = Vgs_sim[idx_above[0]]
    else:
        Vth_const = np.nan
    
    # Calculate transconductance
    gm_max = np.max(gm) * 1e6  # μS
    
    # Calculate subthreshold swing
    # SS = dVgs / d(log10(Id))
    mask = (Id_sim > 1e-12) & (Id_sim < 1e-7)
    if np.sum(mask) > 2:
        Vgs_sub = Vgs_sim[mask]
        Id_sub = Id_sim[mask]
        slope_sub = np.polyfit(np.log10(Id_sub), Vgs_sub, 1)[0]
        SS = slope_sub * 1000  # mV/decade
    else:
        SS = np.nan
    
    print("Extracted MOSFET Parameters:")
    print("="*40)
    print(f"Threshold voltage (linear extrap): Vth = {Vth_linear:.3f} V")
    print(f"Threshold voltage (constant I): Vth = {Vth_const:.3f} V")
    print(f"Peak transconductance: gm_max = {gm_max:.2f} μS")
    print(f"Subthreshold swing: SS = {SS:.1f} mV/decade")
    print(f"\nNote: Ideal SS at 300K = 60 mV/decade")
    
except Exception as e:
    print(f"Could not extract parameters: {e}")

---

## 4. Output Characteristics (Id-Vd)

The output characteristic shows drain current vs drain voltage at different gate voltages.

In [ ]:
# Create MOSFET with output characteristic sweep
sim_idvd = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    junction_depth=0.02,
    device_width=0.125,
    device_depth=0.068,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='nmos',
    
    temperature=300,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="idvd",
    
    # Output characteristic: sweep Vds at fixed Vgs
    vds_sweep=(0.0, 1.5, 0.05),  # Vds: 0 to 1.5V
    vgs=1.0                       # Vgs = 1.0V
)

print("Output Characteristic Simulation Configuration:")
print("="*40)
print("Drain voltage sweep: 0V to 1.5V")
print("Gate voltage: 1.0V (fixed)")

In [ ]:
# Run the output characteristic simulation
print("Running output characteristic simulation...")
result_idvd = sim_idvd.run()

if result_idvd.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")

In [ ]:
# Plot the simulated output characteristic
try:
    iv_data = sim_idvd.get_iv_data()
    Vds_sim, Id_out = iv_data.get_output_characteristic(drain_electrode=2)
    
    Vds_sim = np.array(Vds_sim)
    Id_out = np.array(Id_out)
    
    plt.figure(figsize=(10, 6))
    plt.plot(Vds_sim, Id_out * 1e6, 'b-o', linewidth=2, markersize=4, label='Vgs = 1.0V')
    
    plt.xlabel('Drain Voltage Vds (V)', fontsize=12)
    plt.ylabel('Drain Current Id (μA)', fontsize=12)
    plt.title('NMOS Output Characteristic (PADRE Simulation)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 1.5)
    plt.ylim(0, None)
    
    # Mark linear and saturation regions
    Vth_approx = 0.4  # Approximate threshold
    Vdsat = 1.0 - Vth_approx
    plt.axvline(x=Vdsat, color='r', linestyle='--', alpha=0.5, label=f'Vdsat ≈ {Vdsat:.1f}V')
    plt.annotate('Linear', xy=(0.15, Id_out.max()*0.5e6), fontsize=10)
    plt.annotate('Saturation', xy=(1.0, Id_out.max()*0.5e6), fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Could not plot output characteristic: {e}")

In [ ]:
# Simulate output characteristics at multiple gate voltages
Vgs_values = [0.6, 0.8, 1.0, 1.2]
output_results = {}

print("Simulating output characteristics at multiple Vgs:")
print("="*50)

for Vgs in Vgs_values:
    print(f"\nVgs = {Vgs}V...")
    
    sim = create_mosfet(
        channel_length=0.1,
        gate_oxide_thickness=0.005,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"idvd_vgs{int(Vgs*10)}",
        vds_sweep=(0.0, 1.5, 0.05),
        vgs=Vgs
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vds, Id = iv.get_output_characteristic(drain_electrode=2)
            output_results[Vgs] = (np.array(Vds), np.array(Id))
            print(f"  Success")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot family of output characteristics
if output_results:
    plt.figure(figsize=(10, 6))
    
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    
    for i, (Vgs, (Vds, Id)) in enumerate(output_results.items()):
        color = colors[i % len(colors)]
        plt.plot(Vds, Id * 1e6, color=color, linewidth=2, label=f'Vgs = {Vgs}V')
    
    plt.xlabel('Drain Voltage Vds (V)', fontsize=12)
    plt.ylabel('Drain Current Id (μA)', fontsize=12)
    plt.title('NMOS Output Characteristics (PADRE Simulation)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 1.5)
    plt.ylim(0, None)
    
    plt.tight_layout()
    plt.show()
    
    print("\nOutput characteristic analysis:")
    print("- Linear region: Id increases with Vds")
    print("- Saturation region: Id relatively constant (channel pinch-off)")
    print("- Higher Vgs → Higher saturation current")

---

## 5. Channel Length Effects

Let's explore how channel length affects transistor behavior using PADRE simulations.

In [ ]:
# Compare different channel lengths
channel_lengths = [0.2, 0.1, 0.05]  # microns
length_results = {}

print("Channel Length Comparison:")
print("="*50)

for L_um in channel_lengths:
    print(f"\nL = {L_um*1000:.0f} nm...")
    
    sim = create_mosfet(
        channel_length=L_um,
        gate_oxide_thickness=0.005,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"idvg_L{int(L_um*1000)}nm",
        vgs_sweep=(0.0, 1.5, 0.05),
        vds=0.1
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vgs, Id = iv.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
            length_results[L_um] = (np.array(Vgs), np.array(Id))
            print(f"  Success")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot channel length comparison
if length_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    colors = ['blue', 'green', 'red']
    
    for i, (L_um, (Vgs, Id)) in enumerate(length_results.items()):
        color = colors[i % len(colors)]
        label = f'L = {L_um*1000:.0f} nm'
        ax1.plot(Vgs, Id * 1e6, color=color, linewidth=2, label=label)
        ax2.semilogy(Vgs, Id, color=color, linewidth=2, label=label)
    
    ax1.set_xlabel('Vgs (V)', fontsize=12)
    ax1.set_ylabel('Id (μA)', fontsize=12)
    ax1.set_title('Transfer Characteristic (Linear)', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, 1.5)
    
    ax2.set_xlabel('Vgs (V)', fontsize=12)
    ax2.set_ylabel('Id (A)', fontsize=12)
    ax2.set_title('Transfer Characteristic (Log)', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, 1.5)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey observations:")
    print("- Shorter channel → Higher current (Id ~ 1/L)")
    print("- Short-channel effects may reduce Vth (DIBL)")
    print("- Subthreshold slope may degrade at very short L")

---

## 6. DIBL Analysis (Drain-Induced Barrier Lowering)

DIBL is a short-channel effect where the drain voltage affects the threshold voltage.

In [ ]:
# Simulate at low and high Vds to measure DIBL
Vds_low = 0.05
Vds_high = 1.0
dibl_results = {}

print("DIBL Analysis:")
print("="*50)

for Vds in [Vds_low, Vds_high]:
    print(f"\nVds = {Vds}V...")
    
    sim = create_mosfet(
        channel_length=0.1,
        gate_oxide_thickness=0.005,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"dibl_vds{int(Vds*100)}",
        vgs_sweep=(0.0, 1.5, 0.02),
        vds=Vds
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vgs, Id = iv.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
            dibl_results[Vds] = (np.array(Vgs), np.array(Id))
            print(f"  Success")
        except:
            print(f"  Could not parse data")

In [ ]:
# Plot DIBL comparison
if len(dibl_results) == 2:
    plt.figure(figsize=(10, 6))
    
    Vgs_low, Id_low = dibl_results[Vds_low]
    Vgs_high, Id_high = dibl_results[Vds_high]
    
    plt.semilogy(Vgs_low, Id_low, 'b-', linewidth=2, label=f'Vds = {Vds_low}V')
    plt.semilogy(Vgs_high, Id_high, 'r-', linewidth=2, label=f'Vds = {Vds_high}V')
    
    plt.xlabel('Gate Voltage Vgs (V)', fontsize=12)
    plt.ylabel('Drain Current Id (A)', fontsize=12)
    plt.title('DIBL Analysis (PADRE Simulation)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 1.5)
    
    plt.tight_layout()
    plt.show()
    
    # Extract Vth at constant current and calculate DIBL
    I_th = 1e-7  # 100 nA
    
    idx_low = np.where(Id_low > I_th)[0]
    idx_high = np.where(Id_high > I_th)[0]
    
    if len(idx_low) > 0 and len(idx_high) > 0:
        Vth_low = Vgs_low[idx_low[0]]
        Vth_high = Vgs_high[idx_high[0]]
        
        delta_Vth = Vth_low - Vth_high
        delta_Vds = Vds_high - Vds_low
        DIBL = delta_Vth / delta_Vds * 1000  # mV/V
        
        print(f"\nDIBL Analysis Results:")
        print("="*40)
        print(f"Vth at Vds={Vds_low}V: {Vth_low:.3f} V")
        print(f"Vth at Vds={Vds_high}V: {Vth_high:.3f} V")
        print(f"ΔVth = {delta_Vth*1000:.1f} mV")
        print(f"DIBL = {DIBL:.1f} mV/V")
        print(f"\nNote: DIBL < 100 mV/V is typically acceptable")

---

## 7. Oxide Thickness Effect

Gate oxide thickness affects Cox, Vth, and overall device performance.

In [ ]:
# Compare different oxide thicknesses
tox_values = [0.002, 0.005, 0.010]  # 2nm, 5nm, 10nm
tox_results = {}

print("Oxide Thickness Comparison:")
print("="*50)

for tox in tox_values:
    print(f"\ntox = {tox*1000:.0f} nm...")
    Cox_calc = eps_ox / (tox * 1e-4)  # F/cm^2
    print(f"  Cox = {Cox_calc*1e6:.2f} μF/cm²")
    
    sim = create_mosfet(
        channel_length=0.1,
        gate_oxide_thickness=tox,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"tox_{int(tox*1000)}nm",
        vgs_sweep=(0.0, 1.5, 0.05),
        vds=0.1
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vgs, Id = iv.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
            tox_results[tox] = (np.array(Vgs), np.array(Id))
            print(f"  Simulation success")
        except:
            print(f"  Could not parse data")

In [ ]:
# Plot oxide thickness comparison
if tox_results:
    plt.figure(figsize=(10, 6))
    
    colors = ['blue', 'green', 'red']
    
    for i, (tox, (Vgs, Id)) in enumerate(tox_results.items()):
        color = colors[i % len(colors)]
        label = f'tox = {tox*1000:.0f} nm'
        plt.plot(Vgs, Id * 1e6, color=color, linewidth=2, label=label)
    
    plt.xlabel('Gate Voltage Vgs (V)', fontsize=12)
    plt.ylabel('Drain Current Id (μA)', fontsize=12)
    plt.title('Effect of Oxide Thickness (PADRE Simulation)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 1.5)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey observations:")
    print("- Thinner oxide → Higher Cox → Higher current")
    print("- Thinner oxide → Better gate control")
    print("- Trade-off: Thinner oxide increases gate leakage")

---

## 8. Exercises

### Exercise 1: PMOS Transistor
Create a PMOS transistor and compare its characteristics to the NMOS.

In [ ]:
# Exercise 1: PMOS simulation
sim_pmos = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='pmos',  # PMOS
    temperature=300,
    log_iv=True,
    iv_file="pmos_idvg",
    vgs_sweep=(0.0, -1.5, -0.05),  # Negative voltages for PMOS
    vds=-0.1                        # Negative Vds for PMOS
)

print("Running PMOS simulation...")
result_pmos = sim_pmos.run()

if result_pmos.returncode == 0:
    print("PMOS simulation completed!")
    
    try:
        iv_pmos = sim_pmos.get_iv_data()
        Vgs_p, Id_p = iv_pmos.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
        
        plt.figure(figsize=(10, 6))
        plt.plot(np.abs(Vgs_p), np.abs(Id_p) * 1e6, 'r-o', linewidth=2, label='PMOS')
        
        plt.xlabel('|Vgs| (V)', fontsize=12)
        plt.ylabel('|Id| (μA)', fontsize=12)
        plt.title('PMOS Transfer Characteristic', fontsize=14)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("\nPMOS vs NMOS:")
        print("- PMOS uses holes as carriers (lower mobility)")
        print("- PMOS current ~2-3x lower than NMOS at same |Vgs|")
        print("- PMOS turns on with negative Vgs")
        
    except Exception as e:
        print(f"Could not plot: {e}")
else:
    print("Simulation failed")

---

## Summary

In this notebook, you learned:

1. **MOSFET Structure**: Source, drain, gate, and channel regions
2. **Running PADRE Simulations**: Using `create_mosfet()` and `sim.run()`
3. **Transfer Characteristics**: Id vs Vgs from PADRE simulation
4. **Output Characteristics**: Linear and saturation regions
5. **Parameter Extraction**: Vth, gm, SS from simulated data
6. **Short-Channel Effects**: DIBL analysis from simulation
7. **Design Parameters**: Effect of channel length and oxide thickness

**Key Equations:**
- Linear region: $I_d = \mu C_{ox}\frac{W}{L}[(V_{gs}-V_{th})V_{ds} - \frac{V_{ds}^2}{2}]$
- Saturation: $I_d = \frac{1}{2}\mu C_{ox}\frac{W}{L}(V_{gs}-V_{th})^2$

**Next**: [05 - BJT](05_BJT.ipynb) - Bipolar Junction Transistors